# Data Preprocessing

### lowerCase

In [5]:
def toLower(text):
    return text.lower()

### Numbers to words

In [6]:
import inflect
p = inflect.engine()

import re
reg = r'([0-9]+)'

def isFLoat(strNum):
    try:
        float(strNum)
        return True
    except:
        return False


def converteNumbers(text):
    tempText = text.split()
    newText = []
    for word in tempText:
        tempList = re.split(reg,word)
        for miniWord in tempList:
            if miniWord.isdigit() or isFLoat(miniWord):
                temp = p.number_to_words(miniWord)
                newText.append(removePunctuation(temp))
            else:
                newText.append(miniWord)        
    tempText = ' '.join(newText)
    return tempText

### remove punctuations

In [7]:
import string
translator = str.maketrans(string.punctuation,' '*len(string.punctuation))
def removePunctuation(text):
    global translator
    return text.translate(translator)

### remove WhiteSpaces

In [8]:
def removeWhiteSpace(text):
    return " ".join(text.split())

### remove Stop Words

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def removeStopWords(text):
    sw = set(stopwords.words("english"))
    wt = word_tokenize(text)
    filteredText = [word for word in wt if word not in sw]
    return ' '.join(filteredText)

### Stemming

In [10]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()

def stemWords(text):
    global stemmer
    wt = word_tokenize(text)
    stems = [stemmer.stem(word) for word in wt]
    return ' '.join(stems)

### lemmatization

In [11]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

def lemmatizeWords(text):
    wt = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(word, pos='v') for word in wt]
    return ' '.join(lemmas)

### data preprocessing

#### CACM.all

In [ ]:
import pandas as pd
import re

caRegex = r'CA[0-9a-z]* ?[A-Z]+ '
def TitlePreProcesse(t):
    tempText = toLower(t)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText

def abstractPreProcesse(a):
    tempText = toLower(a)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText

def publicationPreProcesse(p):
    # tempText = toLower(p)
    # tempText = converteNumbers(tempText)
    # tempText = removePunctuation(tempText)
    # tempText = removeWhiteSpace(tempText)
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)

    # TODO remove cacm word and coverte the rest of the data to dateType 
    tempText = p.replace('CACM ','')
    return pd.to_datetime(tempText)

def authorPreProcesse(a):
    tempText = toLower(a)
    # tempText = converteNumbers(tempText)
    
    # TODO take the first wordbefore ,
    lis = tempText.split(' ')
    names = ' '
    l = []
    for word in lis:
      if ',' in word:
          l.append(removePunctuation(word.replace(',','')))
    names = ' '.join(l)

    # tempText = removePunctuation(tempText)
    
    # tempText = removeWhiteSpace(lis[0])
    
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)
    return names

def kPreProcesse(k):
    tempText = toLower(k)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText

def cPreProcesse(c):
    # tempText = toLower(c)
    # tempText = converteNumbers(tempText)
    # tempText = removePunctuation(tempText)
    tempText = removeWhiteSpace(c)
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)
    return tempText

def infoPreProcesse(i):
    # tempText = toLower(i)
    # tempText = converteNumbers(tempText)
    # tempText = removePunctuation(tempText)
    #TODO remove the (CA581202 JB) ... and converte the rest to date type
    tempText = re.sub(caRegex, '', i)
    tempText = pd.to_datetime(tempText)
    #tempText = removeWhiteSpace(tempText)
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)
    return tempText

In [ ]:
a = abstractPreProcesse("""
 Algorithms for parallel computation, and especially comparisons
between parallel and sequential algorithms. Mark W. Krentel
 """)
a

In [ ]:
import pandas as pd
def preprocessedData(dataFrame:pd.DataFrame):
    pdataFrame = pd.DataFrame(columns=['.I','data','.B','.C','.N','.X'])
    seriesDict:dict = {'.I':None, 'data':None, '.B':None, '.C':None,'.N':None, '.X':None} 
    seriesData = seriesDict.copy()
    for i in dataFrame.index:
        try:
            templist = []
            tempC = tempB = tempI = None
            if not dataFrame.loc[i, '.T'] == '':
                templist.append(TitlePreProcesse(dataFrame.loc[i, '.T']))
            if not dataFrame.loc[i, '.W'] == '':
                templist.append(abstractPreProcesse(dataFrame.loc[i, '.W']))
            if not dataFrame.loc[i, '.B'] == '':
                tempB = publicationPreProcesse(dataFrame.loc[i, '.B'])
            if not dataFrame.loc[i, '.A'] == '':
                templist.append(authorPreProcesse(dataFrame.loc[i, '.A']))
            if not dataFrame.loc[i, '.K'] == '':
                templist.append(kPreProcesse(dataFrame.loc[i, '.K']))
            if not dataFrame.loc[i, '.C'] == '':
                tempC = cPreProcesse(dataFrame.loc[i, '.C'])
            if not dataFrame.loc[i, '.N'] == '':
                tempI = infoPreProcesse(dataFrame.loc[i, '.N'])

            seriesData['.I'] = i
            seriesData['data'] = ' '.join(templist)
            seriesData['.B'] = tempB
            seriesData['.C'] = tempC
            seriesData['.N'] = tempI
            seriesData['.X'] = dataFrame.loc[i,'.X']
            pdataFrame = pdataFrame.append(seriesData, ignore_index=True)
        except:
            print(i)
            raise 
    
    pdataFrame.set_index('.I', inplace=True)
    pdataFrame.fillna('', inplace=True)
    return pdataFrame


In [ ]:
import pandas as pd
data = pd.read_csv('../../cacmData/cacmDataCleanedV1.csv')
data.set_index('.I', inplace=True)
data.fillna('', inplace=True)
data.head()

In [ ]:
d = preprocessedData(data)
d.head()

In [ ]:
l = d.loc[-5:, 'data'].tolist()
l[-10:]

In [ ]:
d.to_csv('../../cacmData/cacmDataPreprocessed.csv')

#### query.text

In [20]:
import pandas as pd
import re

infoRegex = r'^( ?[0-9]+\.)' 

def qAbstractPreProcesse(a):
    tempText = toLower(a)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText

def qAuthorPreProcesse(a):
    tempText = toLower(a)
    lis = tempText.split(' ')
    names = ' '
    l = []
    for word in lis:
      if ',' in word:
          l.append(removePunctuation(word.replace(',','')))
    names = ' '.join(l)
    return names

def qInfoPreProcesse(i):
    tempText = re.sub(infoRegex, '', i)
    tempText = qAbstractPreProcesse(tempText)
    return tempText

In [21]:
import pandas as pd
def preprocesseQuery(dataFrame:pd.DataFrame):
    pdataFrame = pd.DataFrame(columns=['.I','data'])
    seriesDict:dict = {'.I':None, 'data':None} 
    seriesData = seriesDict.copy()
    for i in dataFrame.index:
        try:
            templist = []
            if not dataFrame.loc[i, '.W'] == '':
                templist.append(qAbstractPreProcesse(dataFrame.loc[i, '.W']))
            if not dataFrame.loc[i, '.A'] == '':
                templist.append(qAuthorPreProcesse(dataFrame.loc[i, '.A']))
            if not dataFrame.loc[i, '.N'] == '':
                templist.append(qInfoPreProcesse(dataFrame.loc[i, '.N']))

            seriesData['.I'] = i
            seriesData['data'] = ' '.join(templist)
            pdataFrame = pdataFrame.append(seriesData, ignore_index=True)
        except:
            print(i)
            raise 
    
    pdataFrame.set_index('.I', inplace=True)
    pdataFrame.fillna('', inplace=True)
    return pdataFrame


In [22]:
import pandas as pd
querydf = pd.read_csv('../../cacmData/cacmQueryCsv.csv')
querydf.set_index('.I', inplace=True)
querydf.fillna('', inplace=True)
querydf.head()

,.W,.A,.N
.I,,,
1,What articles exist which deal with TSS (Time ...,,"1. Richard Alexander, Comp Serv, Langmuir Lab ..."
2,I am interested in articles written either by ...,"Prieve, B. Pooch, U.","2. Richard Alexander, Comp Serv, Langmuir Lab ..."
3,Intermediate languages used in construction of...,,"3. Donna Bergmark, Comp Serv, Uris Hall (inter..."
4,I'm interested in mechanisms for communicating...,,4. Pavel Curtis (comm mech for disjoint proces...
5,I'd like papers on design and implementation o...,,5. Pavel Curtis (editing interfaces)


In [23]:
preprocessedQuery = preprocesseQuery(querydf)
preprocessedQuery.head()

,data
.I,
1,articl exist deal tss time share system oper s...
2,interest articl write either priev udo pooch p...
3,intermedi languag use construct multi target c...
4,interest mechan commun disjoint process possib...
5,like paper design implement edit interfac wind...


In [24]:
preprocessedQuery.loc[0:1, 'data'].to_list()[0]

'articl exist deal tss time share system oper system ibm comput richard alexand comp serv langmuir lab tss'

In [ ]:
preprocessedQuery.to_csv('../../cacmData/cacmQueryPreprocessed.csv')

#### tests